In [2]:
import numpy as np
import pandas as pd
import requests
import io
from scipy import optimize
import matplotlib.pyplot as plt
import requests, pandas as pd
from bs4 import BeautifulSoup
import urllib.request

In [4]:
density = pd.read_csv(r'C:\Users\vw191\OneDrive\Documents\GitHub\107SBakst.github.io\data\eu_density_data_with_area.csv')

In [6]:
#Global innovation index dataset is from this website. This is data for 2021
#https://innovation-cities.com/indexes

In [33]:
    with open(r'C:\Users\vw191\OneDrive - University of Bristol\Course\3rd year\Data Science\Project\Chart_2_Innovation\INNOVATION_CITIES_INDESX_2021_PREVIEW_2THINKNOW.csv') as f:
        print(f.encoding)

cp1252


In [34]:
global_innovation_index = pd.read_csv(r'C:\Users\vw191\OneDrive - University of Bristol\Course\3rd year\Data Science\Project\Chart_2_Innovation\INNOVATION_CITIES_INDESX_2021_PREVIEW_2THINKNOW.csv', encoding = "cp1252")

In [35]:
global_innovation_index

,2021 Ranking,City,GEOGRAPHY,STATE/PROVINCE,REGION,BLOC,City ID,Current City Classification*,Current Score,Cultural Assets,Human Infrastructure,Networked Markets,Previous 2019 Ranking,Change Since 2019,Pre-Covid 5 Year Average Ranking,Post-COVID 5 year Result
0,-5,Tokyo,Japan,Tokyo,ASIA,JAPAN,1361,NEXUS,56,18,20,18,2,+ 7,4,+ 9
1,-4,Boston,United States,Massachusetts,AMERICAS,USA,167,NEXUS,56,18,19,19,8,+ 12,6,+ 10
2,-3,New York,United States,New York,AMERICAS,USA,901,NEXUS,54,17,17,20,1,+ 4,3,+ 6
3,-2,Sydney,Australia,NSW,ASIA,ANZ,1300,NEXUS,53,19,16,18,15,+ 17,14,+ 16
4,-1,Singapore,Singapore,Singapore,ASIA,ASIA,1240,NEXUS,53,18,17,18,5,+ 6,7,+ 8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,490,Zaporozhye,Ukraine,Zaporizhia Oblast,EMERGING,EURASIA,1518,NaN,27,11,9,7,485,- 5,472,- 18
496,491,Douala,Cameroon,Cameroon Littoral Region,EMERGING,AFRICA,350,NaN,27,10,8,9,495,+ 4,496,+ 5
497,492,Madurai,India,Tamil Nadu,ASIA,INDIA,1545,NaN,27,11,8,8,466,- 26,454,- 38
498,493,La Paz,Bolivia,La Paz Department,AMERICAS,LATIN AMERICA,702,NaN,26,10,7,9,472,- 21,479,- 14
